In [1]:
from kinodata.data.patch_with_data_source import patch_pyg_kinodata3d_dataset
from kinodata.data import KinodataDocked

In [2]:
dataset = KinodataDocked()

In [3]:
# during dataset creation, this data frame is the main data source
df = dataset.df

Reading data frame from /Users/joschkagross/projects/kinodata-3D-affinity-prediction/data/raw/kinodata_docked_v2.sdf.gz...
Deduping data frame (current size: 121913)...
119713 complexes remain after deduplication.
Checking for missing pocket mol2 files...


100%|██████████| 3244/3244 [00:00<00:00, 202752.61it/s]


Adding pocket sequences...


100%|██████████| 119713/119713 [00:00<00:00, 5138910.99it/s]

Exiting with 3552 cached sequences.


In [4]:
# it includes some additional data columns, not all of which are used by the dataset
df.columns

Index(['docking.posit_probability', 'docking.chemgauss_score',
       'assays.chembl_id', 'target_dictionary.chembl_id',
       'molecule_dictionary.chembl_id', 'molecule_dictionary.max_phase',
       'activities.standard_type', 'activities.standard_units',
       'compound_structures.canonical_smiles',
       'compound_structures.standard_inchi', 'component_sequences.sequence',
       'assays.confidence_score', 'docs.chembl_id', 'docs.year',
       'docs.authors', 'UniprotID', 'similar.klifs_structure_id',
       'similar.fp_similarity', 'ID', 'activities.standard_value',
       'docking.predicted_rmsd', 'molecule', 'activities.activity_id',
       'pocket_mol2_file', 'ident', 'structure.pocket_sequence'],
      dtype='object')

In [5]:
# with this method we can patch the dataset with some column from the data source
# eg the chembl assay id
dataset = patch_pyg_kinodata3d_dataset(dataset, "assays.chembl_id", "chembl_assay_id")

Attempting to patch KinodataDocked(119522) to include source column 'assays.chembl_id' as data.'chembl_assay_id'.
Applying permamnent transform RetrieveAndAddColumn(assays.chembl_id, chembl_assay_id, transform=None) to KinodataDocked(119522)...


100%|██████████| 119522/119522 [00:18<00:00, 6598.43it/s]


Done! Collating transformed data list...


In [6]:
from torch_geometric.loader import DataLoader
loader = DataLoader(dataset, batch_size=4, shuffle=True)
batch = next(iter(loader))
print(batch.chembl_assay_id)

['CHEMBL3707950', 'CHEMBL5115458', 'CHEMBL4739197', 'CHEMBL2150524']


In [8]:
# use chembl activities as a sanity check to verify the mapping using kinodata idents
dataset = patch_pyg_kinodata3d_dataset(dataset, "activities.standard_value", "check_activity")

Attempting to patch KinodataDocked(119522) to include source column 'activities.standard_value' as data.'check_activity'.
Applying permamnent transform RetrieveAndAddColumn(activities.standard_value, check_activity, transform=None) to KinodataDocked(119522)...


100%|██████████| 119522/119522 [00:17<00:00, 6860.80it/s]


Done! Collating transformed data list...


In [9]:
loader = DataLoader(dataset, batch_size=4, shuffle=True)
batch = next(iter(loader))
print(batch.y)
print(batch.check_activity)

tensor([6.7212, 7.7670, 7.2161, 7.6343])
tensor([6.7212, 7.7670, 7.2161, 7.6343], dtype=torch.float64)
